In [57]:
import pandas as pd
import re
import unicodedata

In [58]:
df_shop = pd.read_excel('Товары магазина.xlsx')
df_suppliers = pd.read_excel('Прайсы с телеграма 28.01.xlsx')

In [59]:
def normalize_string(text: str) -> str:
    text = re.sub(r'[^\w\s/+-]', ' ', str(text).strip())
    text = unicodedata.normalize('NFKD', text).lower()
    replacements = {
        r'\bgb\b': 'гб',
        r'\b(\d+)\s*гб\b': r'\1гб',
        r'\s+': ' ',
        r'\s*/\s*': '/',
        r'\s*\+\s*': '+'
    }
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text.strip()

In [60]:
def clean_string(input_str: str) -> str:
    return re.sub(r'[^a-zA-Zа-яА-Я0-9]', '', input_str)

In [61]:
shop_dict_devices = {}
shop_dict_manufacturers = {}

shop_desc = {}

names = []

filter_data = []

updated_names = []
manufacturers = set()
models = set()
rams = set()
device_types = set()
processors = set()
colors = set()
manufacturer_codes = set()
roms = set()
for _, row in df_shop.iterrows():
    name = row['Наименование']
    names.append(name)
    updated_names.append("".join(name.split(" ")).lower())
    manufacturer = str(row['Производитель'])
    model = str(row['Модель'])
    ram = int(row['Оперативная память (Gb)']) if str(row['Оперативная память (Gb)']) != "nan" else 0
    device_type = str(row['Тип аппарата'])
    processor = str(row['Процессор'])
    color = str(row['Цвет'])
    manufacturer_code = str(row['Код производителя'])
    rom = str(row['Встроенная память'])
    
    cleaned_manufacturer = clean_string(manufacturer.lower())
    if cleaned_manufacturer not in manufacturers:
        shop_dict_manufacturers[cleaned_manufacturer] = []
    shop_dict_manufacturers[cleaned_manufacturer].append([name, name.lower()])
    filter_data.append(
        [
            name, 
            cleaned_manufacturer, 
            ''.join(model.lower().replace(cleaned_manufacturer + " ", "").split(' ')).replace('plus', "+"), 
            processor.lower().replace(cleaned_manufacturer + " ", ""),
            ram if ram != 'nan' else 0,
            str(rom) if rom != 'nan' else "", 
            color.lower()
        ]
    )
    manufacturers.add(cleaned_manufacturer)
    models.add(''.join(model.lower().replace(cleaned_manufacturer + " ", "").split(' ')))
    rams.add(ram if ram != 'nan' else 0)
    device_types.add(device_type.lower())
    processors.add(processor.lower().replace(cleaned_manufacturer + " ", ""))
    colors.add(color.lower())
    manufacturer_codes.add(manufacturer_code.lower())
    shop_desc[" ".join([cleaned_manufacturer, ''.join(model.lower().replace(cleaned_manufacturer + " ", "").split(' ')).replace('plus', "+"), str(ram) if ram > 0 else "",  str(rom) if rom != 'nan' else "", color.lower()]).replace('nan', "")] = name
    shop_desc[" ".join([cleaned_manufacturer, ''.join(model.lower().replace(cleaned_manufacturer + " ", "").split(' ')).replace('plus', "+"), processor.lower().replace(cleaned_manufacturer + " ", ""), str(ram) if ram > 0 else "",  str(rom) if rom != 'nan' else "", color.lower()]).replace('nan', "")] = name
    
    shop_dict_devices[name] = {
        'Внешний код': row['Внешний код'],
        'Производитель': cleaned_manufacturer,
        'Характеристики': {
            'Модель': ''.join(model.lower().replace(cleaned_manufacturer + " ", "").split(' ')),
            'Оперативная память (Gb)': ram if ram != 'nan' else 0,
            'Тип аппарата': device_type.lower(),
            'Процессор': processor.lower().replace(cleaned_manufacturer + " ", ""),
            'Цвет': color.lower(),
            'Встроенная память': str(rom) if rom != 'nan' else ""
        },
        'Поставщики': []
    }
    
    if rom.lower() != "nan":
        roms.add(rom.lower())
    

In [62]:
def remove_digits(text: str) -> str:
    return re.sub(r'\d+', '', text)

In [63]:
memory_suffixes = set([remove_digits(rom) for rom in roms])

In [64]:
def crear_string(text:str)->str:
    return re.sub(
        r'[^\w\s/\-+()₽$€₴]',
        '', 
        str(text).strip()
    )

In [65]:
def parse_product_string(text):
    cleaned = crear_string(text)
    price_match = re.search(r'(\d+)\D*$', cleaned)
    if not price_match:
        return None
    price = int(price_match.group(1))
    price_start_idx = price_match.start()
    name = cleaned[:price_start_idx].strip()
    name = re.sub(r'\s{2,}', ' ', name)
    name = re.sub(r'\s*([/+])\s*', r'\1', name)
    return {"name": name, "price": price}

In [66]:
from difflib import get_close_matches


def map_color(input_color: str) -> str:
    en_to_ru = {
        'black': 'черный',
        "midnight": 'черный',
        'white': 'белый',
        'red': 'красный',
        'blue': 'синий',
        'dark-blue': 'темно-синий',
        'green': 'зеленый',
        'dark-green': 'темно-зеленый',
        'yellow': 'желтый',
        'orange': 'оранжевый',
        'purple': 'фиолетовый',
        'pink': 'розовый',
        'brown': 'коричневый',
        'gray': 'серый',
        'grey': 'серый',
        "hazel": "коричневый",
        "porcelain": "серебристый",
        'silver': 'серебро',
        'gold': 'золотой',
        'beige': 'бежевый',
        'lilac': 'сиреневый',
        'coral': 'коралловый',
        'obsidian': 'черный',
        'rose-gold': 'розовое золото',
        'space-gray': 'серый космос',
        'midnight-green': 'темно-зеленый',
        'sky-blue': 'голубой',
        'darkgray': 'серый',
        'lightblue': 'голубой',
        'navyblue': 'темно-синий',
        'navy': 'темно-синий',
        'peony': 'розовый',
        'glow': 'бежевый',
        'aqua': 'синий',
        'lemon': 'желтый',
        'graphite': 'серый',
        'olive': 'зеленый'
    }
    normalized_input = input_color
    if normalized_input in russian_colors:
        return normalized_input
    ru_color = en_to_ru.get(normalized_input.replace('-', ''))
    if ru_color and ru_color in russian_colors:
        return ru_color
    matches = get_close_matches(normalized_input, russian_colors, n=1, cutoff=0.6)
    return matches[0] if matches else 'не определено'


english_colors = [
    'black', 'white', 'red', 'blue', 'dark-blue', 'green', 'dark-green',
    'yellow', 'orange', 'purple', 'pink', 'brown', 'gray', 'grey', 'silver',
    'gold', 'beige', 'lilac', 'coral', 'obsidian', 'rose-gold', 'space-gray',
    'midnight-green', 'sky-blue', 'navy', 'darkgray', 'lightblue', 'charcoal',
    'platinum', 'midnight', 'jet-black', 'graphite', 'burgundy', 'sand',
    'mint', 'lavender', 'ivory', 'pearl', 'bronze', 'copper', 'olive',
    'teal', 'turquoise', 'magenta', 'maroon', 'salmon', 'cream', 'ruby',
    'emerald', 'sapphire', 'amber', "hazel", "porcelain", "peony", "glow", 'aqua', 'lemon'
]

russian_colors = [
        'серый', 'бежевый', 'белый', 'голубой', 'желтый', 
        'зеленый', 'темно-зеленый', 'золото', 'золотой', 
        'коралловый', 'коричневый', 'красный', 'оранжевый', 
        'розовое золото', 'розовый', 'серебристый', 'серебро', 
        'серый космос', 'синий', 'сиреневый', 'темно-синий', 
        'фиолетовый', 'черный', 'обсидиан'
    ]

In [67]:
def find_color(parametr_str):
    if parametr_str in english_colors:
        return map_color(parametr_str)
    if parametr_str in russian_colors:
        return parametr_str
    return None

In [68]:

from difflib import SequenceMatcher
from typing import Tuple, Optional

def find_most_similar(
    target: str, 
    candidates: list[str], 
    min_similarity: float = 0.0
) -> Tuple[Optional[str], float]:

    if not candidates:
        return None, 0.0

    target_norm = target.lower().strip()
    candidates_norm = [c.lower().strip() for c in candidates]

    best_match = None
    best_ratio = min_similarity

    for candidate in candidates_norm:
        ratio = SequenceMatcher(None, target_norm, candidate).ratio()
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = candidate

    if best_match is None:
        return None, 0.0

    original_match = candidates[candidates_norm.index(best_match)]
    return original_match, round(best_ratio * 100, 2)

In [69]:
def find_info_from_desc(desc):
    result_name_splited = desc.split(" ")
    company_name = None
    finded_color = None
    find_color_id = None
    finded_ram = None
    finded_rom = None
    finded_ram_id = None
    finded_rom_id = None
    finded_ram_or_rom = False
    for part_name_ind in range(len(result_name_splited)):
        if result_name_splited[part_name_ind] in manufacturers:
            company_name = part_name_ind
            continue
        if not finded_ram_or_rom:
            for suffics in memory_suffixes:
                if str(result_name_splited[part_name_ind]).endswith(suffics):
                    if "+" in result_name_splited[part_name_ind] and result_name_splited[part_name_ind].count("+") == 1:
                        finded_ram_id = part_name_ind
                        finded_rom_id = part_name_ind
                        finded_ram, finded_rom = str(result_name_splited[part_name_ind]).split('+')
                        finded_ram_or_rom = True
                        break
                    if "/" in result_name_splited[part_name_ind] and result_name_splited[part_name_ind].count("/") == 1:
                        finded_ram_id = part_name_ind
                        finded_rom_id = part_name_ind
                        finded_ram, finded_rom = str(result_name_splited[part_name_ind]).split('/')
                        finded_ram_or_rom = True
                        break
                    if str(result_name_splited[part_name_ind - 1]).isdigit():
                        finded_ram = result_name_splited[part_name_ind - 1]
                        finded_rom = result_name_splited[part_name_ind]
                        finded_ram_or_rom = True
                        break
                    finded_rom = result_name_splited[part_name_ind]
                    finded_ram_or_rom = True
                    break
            if "+" in result_name_splited[part_name_ind] and result_name_splited[part_name_ind].count("+") == 1:
                finded_ram_, finded_rom_ = str(result_name_splited[part_name_ind]).split('+')
                if str(finded_ram_).isdigit() and str(finded_rom_).isdigit():
                    finded_ram_id = part_name_ind
                    finded_rom_id = part_name_ind
                    finded_ram, finded_rom = str(result_name_splited[part_name_ind]).split('+')
                    finded_ram_or_rom = True
                    continue
            if "/" in result_name_splited[part_name_ind] and result_name_splited[part_name_ind].count("/") == 1:
                finded_ram_, finded_rom_ = str(result_name_splited[part_name_ind]).split('/')
                if str(finded_ram_).isdigit() and str(finded_rom_).isdigit():
                    finded_ram_id = part_name_ind
                    finded_rom_id = part_name_ind
                    finded_ram, finded_rom = str(result_name_splited[part_name_ind]).split('/')
                    finded_ram_or_rom = True
                    continue
            if finded_ram_or_rom:
                continue
        find_color_result = find_color(result_name_splited[part_name_ind])
        if find_color_result and find_color_result != "не определено":
            finded_color = find_color_result
            find_color_id = part_name_ind
            if not finded_ram_or_rom:
                finded_rom_id = part_name_ind - 1
                finded_rom = result_name_splited[finded_rom_id]
                finded_ram_or_rom = True
            continue
    finded_name_parts = []
    for part_name_ind in range(len(result_name_splited)):
        if company_name == part_name_ind:
            continue
        if find_color_id == part_name_ind:
            break
        if finded_ram_id == part_name_ind:
            break
        if finded_rom_id == part_name_ind:
            break
        finded_name_parts.append(result_name_splited[part_name_ind])
    model_name = ''.join(finded_name_parts)
    if str(finded_rom) in model_name:
        model_name = model_name.replace(finded_rom, "")
    company_name = result_name_splited[company_name] if not company_name is None else None
    return company_name, model_name, finded_color, finded_ram, finded_rom

In [70]:
def get_shop_model_info(shop_model_name):
    result = shop_dict_devices[shop_model_name]
    print(result)

In [71]:
get_shop_model_info("Планшет Samsung Galaxy Tab S9+ SM-X810 12/256GB Бежевый")

{'Внешний код': '3V3Cv0rUB3TqHxU', 'Производитель': 'samsung', 'Характеристики': {'Модель': 'galaxytabs9plus', 'Оперативная память (Gb)': 12, 'Тип аппарата': 'планшет', 'Процессор': 'snapdragon 8 gen 2', 'Цвет': 'бежевый', 'Встроенная память': '256GB'}, 'Поставщики': []}


In [76]:
suppliers_dict = {}
count = 0
for _, row in list(df_suppliers.iterrows())[100:]:
    parsed_result = parse_product_string(row['прайс'])
    company = row['поставщик']
    if parsed_result:
        position_name, position_price = parsed_result['name'], parsed_result['price']
        position_name = position_name.replace(' PLUS', '+')
        lowered_position_name = position_name.lower()
        lowered_position_name = lowered_position_name.replace(' light', '')
        company_name, model_name, finded_color, finded_ram, finded_rom = find_info_from_desc(lowered_position_name)
        # print(company_name, model_name, finded_ram, finded_rom, finded_color, "|", position_name)
        if [company_name, model_name, finded_ram, finded_rom, finded_color].count(NO)
        if company_name in manufacturers and finded_color:
            # print(company_name, model_name, finded_ram, finded_rom, finded_color,"|", position_name)
            ram_rom = f'{finded_ram} {finded_rom}' if finded_ram and finded_rom else (finded_rom if finded_rom else "")
            c_n = str(company_name if not company_name is None else "")
            m_n = str(model_name if not model_name is None else "")
            f_c = str(finded_color if not finded_color is None else "")
            full_name = c_n + " " + m_n + " " + ram_rom + " " + f_c
            list_shop_descs = list(shop_desc.keys())
            finded_position, score = find_most_similar(full_name, list_shop_descs)
            if score > 80:
                shop_dict_devices[shop_desc[finded_position]]['Поставщики'].append((company, position_price))
                print( position_name,"|", full_name, "|", finded_position, '|',  shop_desc[finded_position],"|", score)
            else:
                finded_company = shop_dict_devices[shop_desc[finded_position]]['Производитель']
                finded_model = shop_dict_devices[shop_desc[finded_position]]['Характеристики']['Модель']
                finded_ram = shop_dict_devices[shop_desc[finded_position]]['Характеристики']['Оперативная память (Gb)']
                finded_rom = shop_dict_devices[shop_desc[finded_position]]['Характеристики']['Встроенная память']
                finded_color = shop_dict_devices[shop_desc[finded_position]]['Характеристики']['Цвет']
                if 

SyntaxError: expected ':' (382549189.py, line 13)